<h1>MovieLens-100k<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#使用-|A&amp;B|/sqrt(|A-||-B-|)计算余弦距离" data-toc-modified-id="使用-|A&amp;B|/sqrt(|A-||-B-|)计算余弦距离-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>使用 |A&amp;B|/sqrt(|A || B |)计算余弦距离</a></span></li><li><span><a href="#计算余弦距离" data-toc-modified-id="计算余弦距离-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>计算余弦距离</a></span></li><li><span><a href="#相似余弦距离" data-toc-modified-id="相似余弦距离-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>相似余弦距离</a></span></li><li><span><a href="#生成用户评分的数据结构" data-toc-modified-id="生成用户评分的数据结构-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>生成用户评分的数据结构</a></span></li><li><span><a href="#计算与指定用户最相近的邻居" data-toc-modified-id="计算与指定用户最相近的邻居-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>计算与指定用户最相近的邻居</a></span></li><li><span><a href="#获取电影的列表" data-toc-modified-id="获取电影的列表-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>获取电影的列表</a></span></li><li><span><a href="#读取文件" data-toc-modified-id="读取文件-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>读取文件</a></span></li><li><span><a href="#使用UserFC进行推荐" data-toc-modified-id="使用UserFC进行推荐-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>使用UserFC进行推荐</a></span></li><li><span><a href="#解压rating信息" data-toc-modified-id="解压rating信息-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>解压rating信息</a></span></li><li><span><a href="#主程序" data-toc-modified-id="主程序-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>主程序</a></span></li></ul></div>

In [1]:
import math
import sys
from texttable import Texttable

#   使用 |A&B|/sqrt(|A || B |)计算余弦距离

In [2]:
def calcCosDistSpe(user1,user2):
    avg_x=0.0
    avg_y=0.0
    for key in user1:
        avg_x+=key[1]
    avg_x=avg_x/len(user1)
    
    for key in user2:
        avg_y+=key[1]
    avg_y=avg_y/len(user2)
    
    u1_u2=0.0
    for key1 in user1:
        for key2 in user2:
            if key1[1] > avg_x and key2[1]>avg_y and key1[0]==key2[0]:
                u1_u2+=1
    u1u2=len(user1)*len(user2)*1.0
    sx_sy=u1_u2/math.sqrt(u1u2)
    return sx_sy

#   计算余弦距离

In [3]:
def calcCosDist(user1,user2):
    sum_x=0.0
    sum_y=0.0
    sum_xy=0.0
    for key1 in user1:
        for key2 in user2:
            if key1[0]==key2[0] :
                sum_xy+=key1[1]*key2[1]
                sum_y+=key2[1]*key2[1]
                sum_x+=key1[1]*key1[1]
    
    if sum_xy == 0.0 :
        return 0
    sx_sy=math.sqrt(sum_x*sum_y) 
    return sum_xy/sx_sy

#   相似余弦距离

In [4]:
def calcSimlaryCosDist(user1,user2):
    sum_x=0.0
    sum_y=0.0
    sum_xy=0.0
    avg_x=0.0
    avg_y=0.0
    for key in user1:
        avg_x+=key[1]
    avg_x=avg_x/len(user1)
    
    for key in user2:
        avg_y+=key[1]
    avg_y=avg_y/len(user2)
    
    for key1 in user1:
        for key2 in user2:
            if key1[0]==key2[0] :
                sum_xy+=(key1[1]-avg_x)*(key2[1]-avg_y)
                sum_y+=(key2[1]-avg_y)*(key2[1]-avg_y)
        sum_x+=(key1[1]-avg_x)*(key1[1]-avg_x)
    
    if sum_xy == 0.0 :
        return 0
    sx_sy=math.sqrt(sum_x*sum_y) 
    return sum_xy/sx_sy   

#   生成用户评分的数据结构

In [5]:

#   
#   输入:所以数据 [[2,1,5],[2,4,2]...]
#   输出:1.用户打分字典 2.电影字典
#   使用字典，key是用户id，value是用户对电影的评价，
#   rate_dic[2]=[(1,5),(4,2)].... 表示用户2对电影1的评分是5，对电影4的评分是2
#
def createUserRankDic(rates):
    user_rate_dic={}
    item_to_user={}
    for i in rates:
        user_rank=(i[1],i[2])
        if i[0] in user_rate_dic:
            user_rate_dic[i[0]].append(user_rank)
        else:
            user_rate_dic[i[0]]=[user_rank]
            
        if i[1] in item_to_user:
            item_to_user[i[1]].append(i[0])
        else:
            item_to_user[i[1]]=[i[0]]
            
    return user_rate_dic,item_to_user


#   计算与指定用户最相近的邻居

In [6]:

#   输入:指定用户ID，所以用户数据，所以物品数据
#   输出:与指定用户最相邻的邻居列表
#
def calcNearestNeighbor(userid,users_dic,item_dic):
    neighbors=[]
    #neighbors.append(userid)
    for item in users_dic[userid]:
        print(item)
        for neighbor in item_dic[item[0]]:
            if neighbor != userid and neighbor not in neighbors: 
                neighbors.append(neighbor)
            print(neighbor)

      
    neighbors_dist=[]
    for neighbor in neighbors:
        dist=calcSimlaryCosDist(users_dic[userid],users_dic[neighbor])  #calcSimlaryCosDist  calcCosDist calcCosDistSpe
        neighbors_dist.append([dist,neighbor])
    neighbors_dist.sort(reverse=True)
    #print neighbors_dist
    return  neighbors_dist


#   获取电影的列表

In [7]:
def getMoviesList(file_name):
    #print sys.getdefaultencoding()
    movies_contents=readFile(file_name)
    movies_info={}
    for movie in movies_contents:
        movie_info=movie.split("|")
        movies_info[int(movie_info[0])]=movie_info[1:]
    return movies_info

#   读取文件

In [8]:
def readFile(file_name):
    contents_lines=[]
    f=open(file_name,mode="r",errors='ignore')
    contents_lines=f.readlines()
    f.close()
    return contents_lines

#   使用UserFC进行推荐

In [9]:

#   输入：文件名,用户ID,邻居数量
#   输出：推荐的电影ID,输入用户的电影列表,电影对应用户的反序表，邻居列表
#
def recommendByUserFC(file_name,userid,k=5):
    
    #读取文件数据
    test_contents=readFile(file_name)
    
    #文件数据格式化成二维数组 List[[用户id,电影id,电影评分]...] 
    test_rates=getRatingInformation(test_contents)
#   格式：用户id\t硬盘id\t用户rating\t时间
#   输入：数据集合
#   输出:已经解压的排名信息
#
    def getRatingInformation(ratings):
        rates=[]
        for line in ratings:
            rate=line.split("\t")
            rates.append([int(rate[0]),int(rate[1]),int(rate[2])])
        return rates






    #格式化成字典数据 
    #    1.用户字典：dic[用户id]=[(电影id,电影评分)...]
    #    2.电影字典：dic[电影id]=[用户id1,用户id2...]
    test_dic,test_item_to_user=createUserRankDic(test_rates)

#   输入:所以数据 [[2,1,5],[2,4,2]...]
#   输出:1.用户打分字典 2.电影字典
#   使用字典，key是用户id，value是用户对电影的评价，
#   rate_dic[2]=[(1,5),(4,2)].... 表示用户2对电影1的评分是5，对电影4的评分是2
#
    def createUserRankDic(rates):
        user_rate_dic={}
        item_to_user={}
        for i in rates:
            user_rank=(i[1],i[2])
            if i[0] in user_rate_dic:
                user_rate_dic[i[0]].append(user_rank)
            else:
                user_rate_dic[i[0]]=[user_rank]

            if i[1] in item_to_user:
                item_to_user[i[1]].append(i[0])
            else:
                item_to_user[i[1]]=[i[0]]

        return user_rate_dic,item_to_user






    #寻找邻居
    neighbors=calcNearestNeighbor(userid,test_dic,test_item_to_user)[:k]
        
    recommend_dic={}
    for neighbor in neighbors:
        neighbor_user_id=neighbor[1]
        movies=test_dic[neighbor_user_id]
        for movie in movies:
            #print movie
            if movie[0] not in recommend_dic:
                recommend_dic[movie[0]]=neighbor[0]
            else:
                recommend_dic[movie[0]]+=neighbor[0]
    #print len(recommend_dic)
    
    #建立推荐列表
    recommend_list=[]
    for key in recommend_dic:
        #print key
        recommend_list.append([recommend_dic[key],key])
    
    
    recommend_list.sort(reverse=True)
    #print recommend_list
    user_movies = [ i[0] for i in test_dic[userid]]

    return [i[1] for i in recommend_list],user_movies,test_item_to_user,neighbors
    

#   解压rating信息

In [10]:
#   格式：用户id\t硬盘id\t用户rating\t时间
#   输入：数据集合
#   输出:已经解压的排名信息
#
def getRatingInformation(ratings):
    rates=[]
    for line in ratings:
        rate=line.split("\t")
        rates.append([int(rate[0]),int(rate[1]),int(rate[2])])
    return rates


# 主程序

In [11]:
dpath='./data/'

movies = getMoviesList(dpath+"u.item")

In [12]:
type(movies)

dict

In [13]:
print(movies[1])

['Toy Story (1995)', '01-Jan-1995', '', 'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)', '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0\n']


In [14]:
#读取文件数据
file_name = dpath+"u.data"
test_contents = readFile(file_name)

In [15]:
#文件数据格式化成二维数组 List[[用户id,电影id,电影评分]...] 
test_rates=getRatingInformation(test_contents)

In [16]:
type(test_rates)

list

In [17]:
test_rates[:5]

[[196, 242, 3], [186, 302, 3], [22, 377, 1], [244, 51, 2], [166, 346, 1]]

In [18]:
#格式化成字典数据 
#    1.用户字典：dic[用户id]=[(电影id,电影评分)...]
#    2.电影字典：dic[电影id]=[用户id1,用户id2...]
test_dic,test_item_to_user=createUserRankDic(test_rates)

In [19]:
type(test_dic), type(test_item_to_user)

(dict, dict)

In [20]:
test_dic[22][:5]#用户看过哪些电影,评分是多少

[(377, 1), (376, 3), (128, 5), (80, 4), (241, 3)]

In [21]:
test_item_to_user[377]#该电影被哪些用户看过

[22, 5, 314, 13, 222, 393, 405, 468, 846, 642, 798, 705, 648]

In [22]:
#寻找邻居
userid=179
k=80
neighbors = calcNearestNeighbor(userid,test_dic,test_item_to_user)[:k]

(339, 1)
36
179
147
261
145
284
129
134
409
416
112
13
3
286
191
592
587
688
519
33
666
223
379
682
829
489
39
803
616
782
532
423
863
324
589
780
796
894
422
873
697
909
178
810
404
851
860
(905, 4)
126
179
328
116
100
110
450
334
531
393
193
90
353
174
587
624
752
548
410
515
831
872
13
711
871
782
894
(751, 1)
133
191
195
116
189
179
190
247
41
60
70
176
117
11
224
359
276
258
147
322
396
104
377
293
145
43
294
210
110
33
134
126
311
234
444
351
166
99
493
440
425
201
423
435
178
450
528
509
380
441
393
280
610
384
250
305
472
85
279
125
319
588
650
592
284
676
112
524
144
502
698
708
611
561
640
710
479
531
397
344
164
697
734
229
463
546
320
548
526
717
796
551
369
545
240
738
661
489
64
794
13
807
83
102
603
757
899
488
871
61
818
521
758
30
718
655
454
863
784
898
401
119
894
940
674
800
842
26
625
363
646
729
739
507
199
328
845
727
846
910
751
702
598
221
752
808
105
724
834
297
828
822
830
782
851
253
589
874
578
654
879
519
679
917
100
408
905
841
772
621
197
701
130
813
547

787
423
808
537
404
856
610
827
783
619
840
724
871
453
676
713
718
90
873
450
775
913
395
753
548
681
752
784
758
836
838
919
813
942
831
611
915
863
609
624
675
931
883
592
574
894
646
(303, 1)
302
299
79
4
220
129
87
144
13
40
139
2
219
360
343
329
179
309
112
273
344
211
393
176
204
90
15
113
80
235
448
451
446
296
427
533
186
111
570
3
136
460
592
587
613
334
611
433
655
201
322
579
381
485
6
171
673
354
615
345
539
284
641
392
656
747
413
116
473
173
409
758
683
543
574
784
31
616
547
721
181
860
743
863
819
489
707
315
701
428
770
47
854
931
229
750
676
306
803
149
276
608
926
840
29
568
942
675
405
486
717
410
818
828
503
851
126
430
567
894
937
416
475
431
889
866
842
919
537
934
845
710
293
140
(346, 3)
166
271
197
94
293
71
125
99
130
287
145
235
294
156
363
284
179
146
269
49
104
214
288
116
299
334
133
126
392
458
511
416
474
526
546
414
285
241
385
561
425
624
445
142
542
631
221
456
646
353
171
206
665
149
397
201
418
702
276
141
466
532
655
13
657
656
369
319
724
54
783

521
507
444
515
1
557
393
127
379
592
498
624
290
146
589
327
555
360
61
493
3
499
350
209
392
529
655
448
472
676
610
544
603
33
301
587
699
31
708
727
752
750
104
803
807
717
758
408
428
64
621
724
668
303
784
62
808
877
788
796
281
506
787
916
783
502
567
838
755
853
757
26
145
841
890
851
463
820
896
682
580
929
843
683
889
828
537
435
550
818
259
749
570
862
615
627
72
865
748
702
302
806
813
650
826
396
179
560
805
833
854
545
548
143
459
710
582
526
738
898
697
422
740
816
846
831
100
919
445
117
922
863
119
669
546
902
940
629
660
665
801
509
894
772
871
133
883
907
479
782
(347, 3)
90
197
287
94
229
219
305
276
58
49
112
104
278
299
146
116
363
339
40
143
335
293
163
174
61
39
384
176
334
463
294
284
499
423
356
416
13
475
43
480
485
408
436
410
587
434
100
592
428
588
547
405
393
461
297
655
546
624
271
166
656
673
273
487
646
758
669
385
640
616
574
675
121
752
827
145
713
660
573
787
828
611
704
747
598
515
831
880
392
696
834
3
130
919
860
450
920
724
942
708
753
782
328
7

In [23]:
type(neighbors)

list

In [24]:
neighbors[0:5]

[[0.4650110481141955, 587],
 [0.45488258019294875, 43],
 [0.4529935137611459, 782],
 [0.42936987851534275, 588],
 [0.38879567730637166, 393]]

In [25]:
recommend_dic={}
for neighbor in neighbors:
    neighbor_user_id=neighbor[1]
    movies2=test_dic[neighbor_user_id]
    for movie in movies2:
        #print movie
        if movie[0] not in recommend_dic:
            recommend_dic[movie[0]]=neighbor[0]
        else:
            recommend_dic[movie[0]]+=neighbor[0]
#print len(recommend_dic)

In [26]:
recommend_dic[22], recommend_dic[337]

(7.431989222553849, 0.7063701129234934)

In [27]:
#建立推荐列表
recommend_list=[]
for key in recommend_dic:
    #print key
    recommend_list.append([recommend_dic[key],key])
    
    
recommend_list.sort(reverse=True)
#print recommend_list
user_movies = [ i[0] for i in test_dic[userid]]

In [28]:
recommend_list[0:10]

[[19.689254409700748, 258],
 [18.909639613595868, 288],
 [16.459832104435147, 294],
 [15.743645382058284, 748],
 [15.703008639982286, 286],
 [15.32680086406527, 313],
 [14.980116412792064, 300],
 [14.522258489564877, 50],
 [14.5093766916753, 328],
 [13.646776949006679, 1]]

In [29]:
print(user_movies)

[339, 905, 751, 258, 316, 895, 345, 315, 354, 305, 1316, 269, 301, 310, 272, 750, 303, 346, 331, 893, 321, 302, 917, 690, 307, 333, 362, 340, 288, 300, 1127, 691, 914, 902, 1234, 682, 916, 353, 538, 271, 347, 313]


In [30]:
for m in recommend_list[:20]:
    print(movies[m[1]][0])

Contact (1997)
Scream (1996)
Liar Liar (1997)
Saint, The (1997)
English Patient, The (1996)
Titanic (1997)
Air Force One (1997)
Star Wars (1977)
Conspiracy Theory (1997)
Toy Story (1995)
Fargo (1996)
Return of the Jedi (1983)
Good Will Hunting (1997)
Game, The (1997)
Rock, The (1996)
Twelve Monkeys (1995)
Jerry Maguire (1996)
Devil's Own, The (1997)
Ransom (1996)
Men in Black (1997)


In [31]:
recommend_list = [i[1] for i in recommend_list]
user_movie = user_movies
items_movie = test_item_to_user
neighbors = neighbors
neighbors_id=[i[1] for i in neighbors]

In [32]:
table = Texttable()
table.set_deco(Texttable.HEADER)
table.set_cols_dtype(['t',  # text 
                      't',  # float (decimal)
                      't']) # automatic
table.set_cols_align(["l", "l", "l"])

In [33]:
rows=[]
rows.append([u"movie name",u"release", u"from userid"])
for movie_id in recommend_list[:20]:
    from_user=[]
    for user_id in items_movie[movie_id]:
        if user_id in neighbors_id:
            from_user.append(user_id)
    rows.append([movies[movie_id][0],movies[movie_id][1],""])
table.add_rows(rows)

In [34]:
print(table.draw())

        movie name              release     from userid
Contact (1997)                11-Jul-1997              
Scream (1996)                 20-Dec-1996              
Liar Liar (1997)              21-Mar-1997              
Saint, The (1997)             14-Mar-1997              
English Patient, The (1996)   15-Nov-1996              
Titanic (1997)                01-Jan-1997              
Air Force One (1997)          01-Jan-1997              
Star Wars (1977)              01-Jan-1977              
Conspiracy Theory (1997)      08-Aug-1997              
Toy Story (1995)              01-Jan-1995              
Fargo (1996)                  14-Feb-1997              
Return of the Jedi (1983)     14-Mar-1997              
Good Will Hunting (1997)      01-Jan-1997              
Game, The (1997)              01-Jan-1997              
Rock, The (1996)              07-Jun-1996              
Twelve Monkeys (1995)         01-Jan-1995              
Jerry Maguire (1996)          13-Dec-1996       